In [ ]:
from _init import *

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
import random, torch, requests, json, time

from ranger.utils import json_utils

In [ ]:
def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    print(f'set_seed() seed : {seed}')

seed = 42
set_seed(seed)

In [ ]:
def datas_shuffle(datas: list, seed: int):
    rng = random.Random(seed)
    rng.shuffle(datas)


def load_datas(train_data_path: str, test_data_path: str, seed: int, do_print=False):
    train_datas = json_utils.load_file(train_data_path)
    test_datas = json_utils.load_file(test_data_path)
    datas_shuffle(train_datas, seed)
    datas_shuffle(test_datas, seed)
    
    return train_datas, test_datas

In [ ]:
work_dir = f'/home/nlpshlee/dev_env/git/repos/ranger'
data_dir = f'{work_dir}/data'
out_dir = f'{work_dir}/output'

train_data_path = f'{data_dir}/custom_musique_train_5000_final.jsonl'
test_data_path = f'{data_dir}/custom_multihopqa_eval_1000.jsonl'
train_datas, test_datas = load_datas(train_data_path, test_data_path, seed, do_print=False)

In [ ]:
def request_chain_generate(datas, batch_size, n_chains, chain_depth, adapter_path=''):
    url = CHAIN_SERVER_CONFIG['url_full_generate']
    headers = {'Content-Type': 'application/json'}

    # 전송할 데이터 구성
    payload = {
        'datas': datas,
        'batch_size': batch_size,
        'n_chains': n_chains,
        'chain_depth': chain_depth,
        'adapter_path': adapter_path
    }
    
    try:
        print(f'# [ChainGenerateClient] Requesting chain generation for {len(datas)} items...')

        # POST 요청 전송
        start_time = time.time()
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        elapsed = time.time() - start_time
        
        if response.status_code == 200:
            res_json = response.json()
            print(f'# [ChainGenerateClient] Success! Received {res_json["count"]} results in {elapsed:.2f}s\n')
            return res_json['results']
        else:
            print(f'# [ChainGenerateClient] Error generate : {response.status_code} - {response.text}\n')
            return None

    except requests.exceptions.ConnectionError:
        print(f'# [ChainGenerateClient] Error : Connection failed. Is the server running at {CHAIN_SERVER_CONFIG["url"]} ?\n')
        return None

In [ ]:
def request_reset():
    url = CHAIN_SERVER_CONFIG['url_full_reset']
    headers = {'Content-Type': 'application/json'}

    try:
        print(f'# [ChainGenerateClient] Requesting chain reset...')

        response = requests.post(url, headers=headers, data=[])

        if response.status_code == 200:
            print(f'# [ChainGenerateClient] Success! Chain generator reset complete.\n')
            return True
        else:
            print(f'# [ChainGenerateClient] Error reset : {response.status_code} - {response.text}\n')
            return False
    
    except requests.exceptions.ConnectionError:
        print(f'# [ChainGenerateClient] Error : Connection failed. Is the server running at {CHAIN_SERVER_CONFIG["url"]} ?\n')
        return False

In [ ]:
# adapter_path = ''
adapter_path = '/home/nlpshlee/dev_env/git/repos/ranger/outputs/test/lora_adapter_2025-12-03-07-13-16/'

In [ ]:
responses = request_chain_generate(train_datas[:10], 4, 5, 5)
request_reset()
print(f'{json_utils.to_str(responses)}')

In [ ]:
responses = request_chain_generate(train_datas[:10], 4, 5, 5, adapter_path)
request_reset()
print(f'{json_utils.to_str(responses)}')